# Player Recommender
Hi there! This Notebook is an extension of "Predicting-Lineup-Performance" so please make sure you understood its main concepts before getting started. 
In this small tutorial, we'll how can we use the previously trained models to suggest specific players for a given scenario, thus creating a simple tool that might be used in the hiring process when building a competitive roster. 

### Data Preparation
As always, the first step is to load all required data: in particular, we'll use the matrix of individual 1x10 feature vectors and the pre-trained models (pkl files, one model per stat and per combination). Note as well that, if we want to check who is the ideal player when fitting in a lineup of N players, we have to load the model trained for N+1 players. 

In [5]:
import numpy as np
import pickle

In [6]:
nPlayers = 3
sFolder = '/Users/arbues/Documents/UCAM/Euroleague-Notebooks/Data/'
# Load Feature Vectors + Corresponding Names
matAdvStats = np.load(sFolder + "Lineup-Combos/FeatVecsFin.npy", allow_pickle=True)
matNames = np.load(sFolder + "Lineup-Combos/MatNamesFin.npy", allow_pickle=True)
# Get rid of team strings
matNames = [x[:-4] for x in matNames]
matNames = list(matNames)

# Load the Model of the desired Stat
statString = 'OER'
mName = sFolder + 'Models/' + statString + str(nPlayers+1) + '.pkl'
with open(mName, 'rb') as file:
    model = pickle.load(file)

As we did in the Regression Notebook, the first step is find the feature vectors belonging to each player in the lineup; however, we are still not concatenating them into a single one... You'll see why in a bit! 

In [7]:
# Build the feature vector for any lineup
featVecsi = []
offLoad = []

nameLineup = ''
for i in range(0, nPlayers):
    # Introduce the desired players
    if i == 0:
        name = 'PANGOS, KEVIN'
        year = '2017'
    elif i == 1:
        name = 'HOLLINS, AUSTIN'
        year = '2019'
    elif i == 2:
        name = 'THOMAS, WILL'
        year = '2019'
    elif i == 3:
        name = 'PONITKA, MATEUSZ'
        year = '2019'
    nameLineup = nameLineup + name + '(' + year + ')' + ' / '
    # Find stats
    statsPlay = matAdvStats[matNames.index(name + ' ' + year)]
    featVecsi.append(statsPlay)

### Predictive Model - Test
Although it is a brute-force-based test, in order to find the appropriate player to fit in a particular lineup we are gonna predict the desired stat for every single possibility. Once again, before concatenating, we will sort the individual feature vectors in terms of the carried offensive load / player, thus mantaining the same procedure we followed when training the model. 
Once all predictions have been appended in one same vector, we can sort them according to the predicted value and print the best possible candidates... Easy peasy lemon squeezy!

In [8]:
predictions = []
for iP in range(0, len(matNames)):
    # Since we are gonna sort by offensive load, we ensure to use the original feature vector
    featVecsiC = list(np.copy(featVecsi))
    # For a new player, search stats and append them
    statsPlay = matAdvStats[iP]
    featVecsiC.append(statsPlay)
    # Sort by offensive load
    offLoad = np.array(featVecsiC)[:,1]
    sortOr = np.argsort(offLoad)[::-1]
    # Build new final vector
    featVecs = np.zeros(((nPlayers+1) * 10,))
    for iP in range(0, nPlayers+1):
        featVecs[iP * 10:(iP + 1) * 10] = featVecsiC[sortOr[iP]]
    # Predict performance for each possible player and append it
    pred = model.predict(featVecs.reshape(1, -1))
    predictions.append(pred[0])

# Sort names and stats
predictions, matNames = zip(*sorted(zip(predictions,matNames)))
# Ensure not to mess up with defense (the lower, the better)
if statString != 'DER':
    predictions = predictions[::-1]
    matNames = matNames[::-1]

# Print best fits (minYear might be used to limit the potential options only to active players)
iShow = 0
minYear = 2016 # 0
while iShow < 20: # Limited to show 20 options, feel free to change it
    if int(matNames[iP][-4:]) > minYear:
        print(str(iShow+1) + '. ' + matNames[iP] + ' - ' + str(predictions[iP]))
        iP += 1
        iShow += 1
    else:
        iP += 1

1. MILUTINOV, NIKOLA 2019 - 132.15284257333855
2. TARCZEWSKI, KALEB 2017 - 131.0030207944336
3. LARKIN, SHANE 2019 - 130.90427007047003
4. MUHAMMED, ALI 2018 - 130.27235987493296
5. SINGLETON, CHRIS 2017 - 130.11092382364967
6. PUNTER, KEVIN 2019 - 130.03392532864456
7. JOVIC, STEFAN 2019 - 129.8571186940606
8. NUNNALLY, JAMES 2017 - 129.83470577506762
9. TAVARES, WALTER 2018 - 129.77545675295215
10. MAHMUTOGLU, MELIH 2018 - 129.65451545706662
11. GUDAITIS, ARTURAS 2018 - 129.49625358507467
12. GUDURIC, MARKO 2018 - 129.31851437750262
13. ULANOVAS, EDGARAS 2017 - 129.28981766404115
14. VOIGTMANN, JOHANNES 2017 - 129.21018645577044
15. TAVARES, WALTER 2017 - 129.1339784854256
16. MARTINEZ, RAFA 2017 - 128.6107743976535
17. GILL, ANTHONY 2017 - 128.58651652142748
18. MILUTINOV, NIKOLA 2017 - 128.58147430063588
19. GUDURIC, MARKO 2017 - 128.44443105099847
20. WACZYNSKI, ADAM 2017 - 128.34473633721842


Hope you liked it! 
If you have any questions / suggestions, feel free to send me an email (adria.arbues@upf.edu) or a Twitter DM (@arbues6). 